## Import Libraries

In [1]:
from os import path
from itertools import product
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter, MaxNLocator
import scipy.io as sio

from brainpipe.classification import *
from brainpipe.system import study
from brainpipe.feature import power, amplitude, sigfilt
from brainpipe.visual import *
from brainpipe.statistics import *
from scipy.stats import *

## User variables

In [2]:
# PATH TO DATA
bsl=None
st = study('Olfacto')
path_pow = path.join(st.path, 'feature/7_Power_E1E2_Odor_Good_Bad_EpiScore_Expi/')
save_path = path.join(st.path, 'classified/1_Classif_Mean_Power_'+str(bsl)+'_EpiScore_18rois_3wins_expi/MF/')

# POWER & STATS PARAMETERS
nfreq = 8
nperm = 100

-> Olfacto loaded


## Power Decoding - Good Bad Odors Encoding

In [8]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import StratifiedKFold as SKFold
from sklearn.metrics import roc_auc_score
from numpy.random import permutation

phases, bsl, subjects = ['odor'],['None'],['LEFC']
for b, su, phase in product(bsl,subjects,phases):
    #=========================== Load Power files (nfreq, nelec, nwin, ntrial) =================================    
    bad_data = np.load(path.join(path_pow, su+'_'+phase+'_bad_bipo_sel_'+b+'_power.npz'))['xpow']
    good_data = np.load(path.join(path_pow, su+'_'+phase+'_good_bipo_sel_'+b+'_power.npz'))['xpow']
    names = np.load(path.join(path_pow, su+'_'+phase+'_bad_bipo_sel_'+b+'_power.npz'))['labels']
    channels = np.load(path.join(path_pow, su+'_'+phase+'_bad_bipo_sel_'+b+'_power.npz'))['channels']
    freq_names = np.load(path.join(path_pow, su+'_'+phase+'_bad_bipo_sel_'+b+'_power.npz'))['fname']
    print (su, 'bad shape: ', bad_data.shape, 'good shape: ', good_data.shape)

    # =========================== Select Power for all elec 1 freq =================================                 
    for elec_num in range(bad_data.shape[1]):
    #for freq in range(nfreq):
        # 1 freq // Bad-Good conditions (ntrials, nfreq, nwins)
        bad_data_elec = bad_data[[2,5,6],elec_num,:,:].swapaxes(0,1).swapaxes(0,2)
        good_data_elec = good_data[[2,5,6],elec_num,:,:].swapaxes(0,1).swapaxes(0,2)
        print ('data elec ', bad_data_elec.shape, good_data_elec.shape)
        nwin, channel, label = good_data_elec.shape[2], channels[elec_num], names[elec_num]
        
# =============================  Classification Computation ============================================================           
        # create a data matrix, concatenate along the trial dimension
        x = np.concatenate((bad_data_elec, good_data_elec), axis=0)
        print ('Size of the concatenated data: ', x.shape, 'Number time windows : ', x.shape[2])
        #create label vector (0 for rest and 1 for odor)
        y = np.asarray([0]*bad_data_elec.shape[0] + [1]*good_data_elec.shape[0])
        print ('Size of label for classif: ', len(y))

        auc = np.array([])
        for t in range(x.shape[2]):
            X = x[:,:,t]
            #X = X.reshape(-1, 1)
            score_rep = []
            for i in range(10):
                skf = SKFold(n_splits=10, random_state=i)
                skf.get_n_splits(X, y)
                score_cv = []
                for train_index, test_index in skf.split(X, y):
                    clf = LDA()
                    X_train, X_test = X[train_index], X[test_index]
                    y_train, y_test = y[train_index], y[test_index]
                    clf.fit(X=X_train, y=y_train)
                    y_pred = clf.predict(X_test)
                    score_cv.append(roc_auc_score(y_test,y_pred,average='weighted'))
                score_rep.append(np.mean(score_cv))
            score_rep = np.asarray(score_rep).reshape(1,len(score_rep))
            auc = np.vstack((auc, score_rep)) if np.size(auc) else score_rep
        auc = np.swapaxes(auc,0,1)
        print(auc.shape)

        perm_scores = np.array([])
        for t in range(x.shape[2]):
            X = x[:,:,t]
            #X = X.reshape(-1, 1)
            perm_rep = []
            for perm in range(nperm):
                y_perm = y[permutation(len(y))]
                score_cv = []
                for train_index, test_index in skf.split(X, y_perm):
                    clf = LDA()
                    X_train, X_test = X[train_index], X[test_index]
                    y_train, y_test = y_perm[train_index], y_perm[test_index]
                    clf.fit(X=X_train, y=y_train)
                    y_pred = clf.predict(X_test)
                    score_cv.append(roc_auc_score(y_test,y_pred,average='weighted'))
                perm_rep.append(np.mean(score_cv))
            perm_rep = np.asarray(perm_rep).reshape(1,len(perm_rep))
            perm_scores = np.vstack((perm_scores, perm_rep)) if np.size(perm_scores) else perm_rep
            #print(perm_scores.shape)
        perm_scores = np.swapaxes(perm_scores,0,1)

        th_0_05_perm = perm_pvalue2level(perm_scores, p=0.05, maxst=True)
        th_0_01_perm = perm_pvalue2level(perm_scores, p=0.01, maxst=True)
        print('th_perm : ', th_0_05_perm[0], th_0_01_perm[0])

# ========================== Create a pvalue vector for uac measure ========================
        auc_pvals = []
        for i in range(auc.shape[1]):
            if np.mean(auc[:,i]) > th_0_01_perm[0]:
                auc_pvals.append(0.009)
            elif np.mean(auc[:,i]) > th_0_05_perm[0]:
                auc_pvals.append(0.04)
            else:
                auc_pvals.append(1)
        print (auc_pvals)

# ============================== PLOT POWER ANALYSIS + STATS & DECODING ACCURACY ===================================================
        # plot and figure parameters
        xfmt = ScalarFormatter(useMathText=True)
        xfmt.set_powerlimits((0,3))
        fig = plt.figure(1,figsize=(7,5))
        title = 'MF All frequency AUC Score for '+su+' Bad/Good '+str(channel)+' '+str(label)+' ('+str(elec_num)+')'
        fig.suptitle(title, fontsize=12)
        # Time vector to plot power
        step = 5500/ nwin
        times_plot = np.arange(0, 5500, step)
        print(step, len(times_plot))

        # Plot DA for the POW
        BorderPlot(times_plot, auc, color='b', kind='sd',xlabel='Time (ms)', 
                   ylim=[0.4,0.9], ylabel='AUC Score',linewidth=2, alpha=0.3)
        rmaxis(plt.gca(), ['right', 'top'])
        addLines(plt.gca(), vLines=[1300,4000], vWidth=[1.5]*2, vColor=['grey']*2, hLines=[0.5], 
                 hColor=['#000000'], hWidth=[1.5])
        plt.legend(loc=0, handletextpad=0.1, frameon=False)   
        plt.gca().yaxis.set_major_locator(MaxNLocator(3,integer=True))
        plt.plot(times_plot, th_0_05_perm*np.ones(len(times_plot)), '--', color='orange', linewidth=2)
        plt.plot(times_plot, th_0_01_perm*np.ones(len(times_plot)), '--', color='r', linewidth=2)

# ============================== SAVE FIGURE & DECODING ACCURACY =============================
        #Filenames to save
        name_auc = (save_path+su +'_auc_Good_Bad_'+channel+'_'+label+'_('+str(elec_num)+').npy')
        name_th_0_05_perm = (save_path+su +'_th_0_05_perm_'+channel+'_'+label+'_('+str(elec_num)+').npy')
        name_th_0_01_perm = (save_path+su +'_th_0_01_perm_'+channel+'_'+label+'_('+str(elec_num)+').npy')
        plot_name = (save_path+su +'_MF_freq_Good_Bad_'+str(channel)+'_'+str(label)+'_('+str(elec_num)+').png')            
        #Save plots
        np.save(name_auc, auc)
        np.save(name_th_0_05_perm, th_0_05_perm[0])
        np.save(name_th_0_01_perm, th_0_01_perm[0])
        plt.savefig(plot_name, dpi=300, bbox_inches='tight')
        plt.clf()
        plt.close()
        del bad_data_elec, good_data_elec, X, auc
del bad_data, good_data

LEFC bad shape:  (8, 68, 49, 35) good shape:  (8, 68, 49, 23)
data elec  (35, 3, 49) (23, 3, 49)
Size of the concatenated data:  (58, 3, 49) Number time windows :  49
Size of label for classif:  58
(10, 49)
th_perm :  0.65 0.733333333333
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.04, 0.04, 1, 0.04, 0.04, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.04, 0.04, 0.04, 1, 1, 1, 1, 1, 1]
112.24489795918367 49
not legend
data elec  (35, 3, 49) (23, 3, 49)
Size of the concatenated data:  (58, 3, 49) Number time windows :  49
Size of label for classif:  58
(10, 49)
th_perm :  0.65 0.758333333333
[0.009, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.04, 0.04, 0.04, 1, 1, 1, 0.04, 1, 0.04, 1, 0.04, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.04, 0.04, 0.04, 0.04, 0.04, 1, 1]
112.24489795918367 49
not legend
data elec  (35, 3, 49) (23, 3, 49)
Size of the concatenated data:  (58, 3, 49) Number time windows :  49
Size of label for classif:  58
(10, 49)
th_perm :  0.6375 0.7